In [26]:
import pandas as pd
import numpy as np
import re

In [54]:
data = pd.read_csv('/home/guilherme/Documentos/repos/ds_ao_dev/projeto_webscraping/df_raw.csv')

In [55]:
data.head()

,Unnamed: 0,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
0,0,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Shell: Cotton 99%, Spandex 1%"
1,1,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Pocket lining: Polyester 65%, Cotton 35%"
2,2,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Shell: Cotton 99%, Spandex 1%"
3,3,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Pocket lining: Polyester 65%, Cotton 35%"
4,4,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Denim blue,"Shell: Cotton 99%, Spandex 1%"


In [56]:
data = data.drop(['Unnamed: 0' ],axis=1)
data.sample()

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
6084,875105016,men_jeans_relaxed,Relaxed Jeans,29.99,2022-04-24-11-21:30,875105,16,Light gray,Shell: Cotton 100%


In [57]:
#product id

data = data.dropna(subset=['product_id'])

#product category

#product name
data['products_name']= data['products_name'].apply(lambda x: x.replace(' ','_').lower())
#products_price

#já fiz o tratamento anteriomente

#product datetime

data['scrapy_datetime']= pd.to_datetime(data['scrapy_datetime'])
#style id

#color id

#color name
data['color_name']= data['color_name'].apply(lambda x: x.replace(' ','_').replace('/','_').lower() if pd.notnull(x) else x)

#Composition

data['Composition']= data['Composition'].where(~data['Composition'].str.contains('Pocket lining:',na=False) ) 

data['Composition']= data['Composition'].where(lambda x: ~x.str.contains('Shell:',na=False) )

data['Composition'] = data['Composition'].where(lambda x: ~x.str.contains('Lining:',na=False) )


#break composition by comma (separando as composições dos materiais por coluna)
df1 = data['Composition'].str.split(',',expand=True)


#Cotton| Spandex | Polyester |  Elastomultiester

data.head()

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
1,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
2,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
4,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,denim_blue,NaN


In [58]:
#Cotton| Spandex | Polyester |  Elastomultiester
#2 etapa
#definir as colunas (organizando as colunas por máteria-prima)

#criando Dataframe vazio com o mesmo tamanho do dataframe original para prencher com as respectivas composições
df_ref = pd.DataFrame(index=np.arange(len(data) ),columns=['cotton','spandex','polyester','elastomultiester'])

#cotton
#pegando a coluna da composição cotton
df_cotton = df1[0]

#colocando o nome na coluna 
df_cotton.name = 'cotton'
#mesclando com a tabela vazia 
df_ref = pd.concat([df_ref,df_cotton],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#polyester 

#adicionando as coluna da composição Polyester
df_polyester =df1[1].where(lambda x: x.str.contains('Polyester',na=True))

#renomeando a coluna
df_polyester.name= 'polyester'

df_ref= pd.concat([df_ref,df_polyester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#Elastomultiester

#coletando apenas a composição elastano
df_elastomultiester =df1[1].where(lambda x: x.str.contains('Elastomultiester',na=True))

#renomenado a coluna eslastano
df_elastomultiester.name= 'elastomultiester'

#mesclando com df_ref vazio e com a mesma quantidade de linhas que há no index
df_ref= pd.concat([df_ref,df_elastomultiester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#Spandex

#coletando composição spandex da segunda coluna df1

df_spandex_1= df1[1].where(lambda x: x.str.contains('Spandex',na=True))

df_spandex_1.name = 'spandex'

df_ref= pd.concat([df_ref,df_spandex_1],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#coletando composição spandex da terceira coluna do df1

df_spandex_2= df1[2].where(lambda x: x.str.contains('Spandex',na=True))
             
df_spandex_2.name= 'spandex'

df_ref= pd.concat([df_ref,df_spandex_2],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]



#final join

data= pd.concat([data,df_ref],axis=1)

#coletando apenas os numeros e transformando em números inteiros

#format composition data

#cotton
data['cotton']= data['cotton'].apply(lambda x: int(re.search('\d+',x ).group(0))/100 if pd.notnull(x) else x)

#spandex
data['spandex'] =data['spandex'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

#polyester
data['polyester']= data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#elastomultiester
data['elastomultiester']=data['elastomultiester'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

In [60]:
data.sample(18)

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition,cotton,polyester,elastomultiester,spandex
3380,985159008,men_jeans_skinny,skinny_jeans,19.99,2022-04-24 11:00:00-21:30,985159,8,light_denim_blue,NaN,NaN,NaN,NaN,NaN
526,811993036,men_jeans_regular,regular_jeans,29.99,2022-04-24 11:00:00-21:30,811993,36,denim_blue,"Cotton 98%, Spandex 2%",0.98,NaN,NaN,NaN
104,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,denim_blue,NaN,NaN,NaN,NaN,NaN
64,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,dark_denim_blue,NaN,NaN,NaN,NaN,NaN
5600,985197003,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,985197,3,light_denim_blue,NaN,NaN,NaN,NaN,NaN
3777,985159005,men_jeans_skinny,skinny_jeans,19.99,2022-04-24 11:00:00-21:30,985159,5,dark_denim_blue,NaN,NaN,NaN,NaN,NaN
138,1024256002,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,2,white,NaN,NaN,NaN,NaN,NaN
5065,875105024,men_jeans_relaxed,relaxed_jeans,29.99,2022-04-24 11:00:00-21:30,875105,24,light_denim_blue,Cotton 100%,1.00,NaN,NaN,NaN
1551,1024256005,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,5,dark_gray,NaN,NaN,NaN,NaN,NaN
1841,690449056,men_jeans_ripped,skinny_jeans,39.99,2022-04-24 11:00:00-21:30,690449,56,light_denim_blue,"Cotton 98%, Spandex 2%",0.98,NaN,NaN,NaN
